In [4]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qg-WLam-V8ONRTu3xJZEmtYaHlMUk0fO2rrGi78CKsmNWlz3JUK8G0



Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [7]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [8]:
out_dir = os.getcwd()

In [9]:
# read Tree Mosaic Land
tml = ee.ImageCollection('projects/wri-datalab/TML')
tml_tree_cover = tml.reduce(ee.Reducer.mean()).rename('b1')

In [10]:
# select only pixels with more than 10% tree cover
# tree_percent_threshold = 10 
# tml_tree_cover_threshold = tml_tree_cover.updateMask(tml_tree_cover.gte(tree_percent_threshold))

In [11]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 

# subset cities with avaialable tml data
tml_available_cities = ['Teresina city','Teresina region','Belem city','Belem region','San Jose','Kigali',
                        'Freetown city','Freetown region','Marrakech','Chennai region','Chennai region',
                        'Pune','Surat','Jakarta','Bitung','Semarang','Balikpapan','Balikpapan']
boundary_georef = boundary_georef.loc[boundary_georef['city_name'].isin(tml_available_cities)].reset_index(drop=True)
boundary_georef   

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America
1,BRA-Teresina,region,ADM2union,ADM2,Teresina region,Brazil,BRA,America
2,BRA-Belem,city,ADM4union,ADM4,Belem city,Brazil,BRA,America
3,BRA-Belem,region,ADM2union,ADM2,Belem region,Brazil,BRA,America
4,CRI-San_Jose,region,ADM2union,ADM2,San Jose,Costa Rica,CRI,America
5,RWA-Kigali,city,ADM4union,ADM4,Kigali,Rwanda,RWA,Africa
6,SLE-Freetown,city,ADM4cityunion,ADM4city,Freetown city,Sierra Leone,SLE,Africa
7,SLE-Freetown,region,ADM4regionunion,ADM4region,Freetown region,Sierra Leone,SLE,Africa
8,MAR-Marrakech,NaN,ADM2,ADM5,Marrakech,Morocco,MAR,Africa
9,IND-Chennai,region,ADM4union,ADM4,Chennai region,India,IND,Asia


# Compute indicator

In [12]:
cities_indicators_sicb_11 = pd.DataFrame()

In [14]:
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # compute average tml
    tml_mean = tml_tree_cover.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': boundary_geo_ee.geometry(),
        'scale': 10,
        'maxPixels': 1e9
    })
    # get output
    tml_mean_dict = tml_mean.getInfo()
    tml_mean_value = tml_mean_dict['b1']
    
    # store in df and apend
    df = geemap.ee_to_pandas(boundary_geo_ee)
    df['SICB_11_percent_of_tree_cover'] = tml_mean_value
    cities_indicators_sicb_11 = cities_indicators_sicb_11.append(df)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
     # compute average tml
    tml_mean = tml_tree_cover.reduceRegions(**{
        'reducer': ee.Reducer.mean(),
        'collection': boundary_geo_ee,
        'scale': 10
    })
    
    # store output in df and apend
    df = geemap.ee_to_pandas(tml_mean)
    df = df.rename(columns={"mean": "SICB_11_percent_of_tree_cover"})
    cities_indicators_sicb_11 = cities_indicators_sicb_11.append(df)


0

 geo_name: BRA-Teresina

 boundary_id_aoi: BRA-Teresina-ADM4union

 boundary_id_unit: BRA-Teresina-ADM4
1

 geo_name: BRA-Teresina

 boundary_id_aoi: BRA-Teresina-ADM2union

 boundary_id_unit: BRA-Teresina-ADM2
2

 geo_name: BRA-Belem

 boundary_id_aoi: BRA-Belem-ADM4union

 boundary_id_unit: BRA-Belem-ADM4
3

 geo_name: BRA-Belem

 boundary_id_aoi: BRA-Belem-ADM2union

 boundary_id_unit: BRA-Belem-ADM2
4

 geo_name: CRI-San_Jose

 boundary_id_aoi: CRI-San_Jose-ADM2union

 boundary_id_unit: CRI-San_Jose-ADM2
5

 geo_name: RWA-Kigali

 boundary_id_aoi: RWA-Kigali-ADM4union

 boundary_id_unit: RWA-Kigali-ADM4
6

 geo_name: SLE-Freetown

 boundary_id_aoi: SLE-Freetown-ADM4cityunion

 boundary_id_unit: SLE-Freetown-ADM4city
7

 geo_name: SLE-Freetown

 boundary_id_aoi: SLE-Freetown-ADM4regionunion

 boundary_id_unit: SLE-Freetown-ADM4region
8

 geo_name: MAR-Marrakech

 boundary_id_aoi: MAR-Marrakech-ADM2

 boundary_id_unit: MAR-Marrakech-ADM5
9

 geo_name: IND-Chennai

 boundary_id_aoi

In [21]:
cities_indicators_sicb_11.head()

,geo_parent_name,geo_level,creation_date,geo_id,geo_name,SICB_11_percent_of_tree_cover
0,BRA-Teresina,ADM-4-union,2022-06-27,BRA-Teresina_ADM-4-union_1,BRA-Teresina,29.863539
0,BRA-Teresina,ADM-4,2022-06-27,BRA-Teresina_ADM-4_1,Parque Juliana,30.893434
1,BRA-Teresina,ADM-4,2022-06-27,BRA-Teresina_ADM-4_2,Parque Jacinta,14.111674
2,BRA-Teresina,ADM-4,2022-06-27,BRA-Teresina_ADM-4_3,Portal da Alegria,21.957233
3,BRA-Teresina,ADM-4,2022-06-27,BRA-Teresina_ADM-4_4,Pedra Miúda,51.458839


# Merge with indicator table

In [16]:
# read indicator table
cities_indicators = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/indicators/cities_indicators.csv') 

In [17]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace by new indicators")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [18]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_sicb_11,
                                            indicator_name = "SICB_11_percent_of_tree_cover")

add new indicators


In [19]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,SICB_11_percent_of_tree_cover
0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza,NaN
1,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza,NaN
2,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza,NaN
3,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza,NaN
4,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza,NaN
...,...,...,...,...,...
1937,IDN-Palembang_ADM-3_12,ADM-3,Bukit Kecil,IDN-Palembang,NaN
1938,IDN-Palembang_ADM-3_13,ADM-3,Ilir Barat II,IDN-Palembang,NaN
1939,IDN-Palembang_ADM-3_14,ADM-3,Ilir Barat I,IDN-Palembang,NaN
1940,IDN-Palembang_ADM-3_15,ADM-3,Gandus,IDN-Palembang,NaN


# Upload in aws s3

In [27]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [28]:
# upload to aws
key_data = 'indicators/cities_indicators.csv'
bucket_name = 'cities-urbanshift' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [29]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')